In [5]:
import lmdb
import os

# 操作 LMDB 的流程

- 通过 `env = lmdb.open()` 打开环境
- 通过 `txn = env.begin()` 建立事务
- 通过 `txn.put(key, value)` 进行插入和修改
- 通过 `txn.delete(key)` 进行删除
- 通过 `txn.get(key)` 进行查询
- 通过 `txn.cursor()` 进行遍历
- 通过 `txn.commit()` 提交更改

In [6]:
root = 'D:/datasets/monkey'

查看当前文件：

In [7]:
os.listdir(root)

['monkey_labels.txt', 'training', 'validation']

In [8]:
env = lmdb.open(root)

查看当前文件变化：

In [9]:
os.listdir(root)

['data.mdb', 'lock.mdb', 'monkey_labels.txt', 'training', 'validation']

多了 `'data.mdb', 'lock.mdb'` 这两个文件。

# 插入、删除、修改

插入与修改都用 `put` 实现，删除用 `delete` 实现。

使用 `env.begin` 创建事务时，只有 `write=True` 才能够写数据库：

In [10]:
txn = env.begin(write=True)

txn.put(b'1', b"Alice")
txn.put(b'2', b"Bob")
txn.put(b'3', b"Peter")

txn.delete(b'1')

txn.put(b'3', b"Mark")

txn.commit()

# 查询

查单条记录用 `get(key)`，遍历数据库用 `cursor`。

In [11]:
txn = env.begin()
print(txn.get(b'2'))

for key, value in txn.cursor():
    print(key, value)

b'Bob'
b'2' b'Bob'
b'3' b'Mark'


In [5]:
import lmdb

class LmdbProgress:
    def __init__(self, root):
        self.root = root
        self.env = lmdb.open(self.root)
        
    def context(self):
        return self.env.begin(write = True) # 创建事务，并写入
        
    def toByte(self, inputs):
        if isinstance(inputs, int):
            return str(inputs).encode()
        elif isinstance(inputs, bytes):
            return inputs
        else:
            return inputs.encode()
        
    def insert(self, sid, name):
        sid = self.toByte(sid)
        name = self.toByte(name)
        txn = self.context()
        txn.put(sid, name)
        txn.commit()
        
    def delete(self, sid):
        txn = self.context()
        sid = self.toByte(sid)
        txn.delete(sid)
        txn.commit()
        
    def update(self, sid, name):
        txn = self.context()
        sid = self.toByte(sid)
        name = self.toByte(name)
        txn.put(sid, name)
        txn.commit()

    def search(self, sid):
        txn = self.env.begin()
        sid = self.toByte(sid)
        name = txn.get(sid)
        return name

    def display(self):
        txn = self.env.begin()
        cur = txn.cursor()
        for key, value in cur:
            print((key, value))
            
    def close(self):
        self.env.close()
        
    def reinit(self):
        '''
        重新打开 lmdb
        '''
        self.env = lmdb.open(self.root)

In [43]:
root = '../data/draft'
db = LmdbProgress(root)  # 初始化一个 db

In [44]:
print("Insert 3 records.")
db.insert(1, "Alice")
db.insert(2, "Bob")
db.insert(3, "Peter")
db.display()

Insert 3 records.
(b'1', b'Alice')
(b'2', b'Bob')
(b'3', b'Peter')


In [45]:
print("Delete the record where sid = 1.")
db.delete(1)
db.display()

Delete the record where sid = 1.
(b'2', b'Bob')
(b'3', b'Peter')


In [46]:
print("Update the record where sid = 3.")
db.update(3, "Mark")
db.display()

Update the record where sid = 3.
(b'2', b'Bob')
(b'3', b'Mark')


In [47]:
print("Get the name of student whose sid = 3.")
name = db.search(3)
print(name)

Get the name of student whose sid = 3.
b'Mark'


关闭 lmdb

In [48]:
db.env.close()

In [49]:
db.reinit() # 再次打开

上面的准备工作已经做完，下面讨论如何将目标检测数据集转换为 lmdb 格式。

In [6]:
root = '../data/draft'
db = LmdbProgress(root)

In [7]:
import os

In [11]:
def make_dir(root, dir_name):
    '''
    在 root 下生成目录
    '''
    _dir = root + dir_name + "/"  # 拼出分完整目录名
    if not os.path.exists(_dir):  # 是否存在目录，如果没有创建
        os.makedirs(_dir)
    return _dir


def get_dir_names(root):
    dir_names = []
    for k in os.listdir(root):
        if os.path.isdir(root + k):  # 判断是否是目录
            dir_names.append(root + k)
    return dir_names

In [ ]:
class DetLmdb:
    def __init__(self, root):
        self.trainX = LmdbProgress(make_dir(root, 'trainX'))
        self.trainYX = LmdbProgress(make_dir(root, 'trainY'))
        self.valX = LmdbProgress(make_dir(root, 'valX'))
        self.valY = LmdbProgress(make_dir(root, 'valY'))

In [ ]:
def img2lmdb():
    # 创建数据库文件
    env = lmdb.open(cfg.dataset, max_dbs=4, map_size=1e12) # map_size 表示最大的存储尺寸
    # 创建对应的数据库
    train_data = env.open_db("train_data")
    train_label = env.open_db("train_label")
    val_data = env.open_db("val_data")
    val_label = env.open_db("val_label")
    train_image_list, train_label_list = get_image_label_list(train=True)
    val_image_list, val_label_list = get_image_label_list(train=False)
    # 把图像数据写入到LMDB中
    with env.begin(write=True) as txn:
        for idx, path in enumerate(train_image_list):
            logging.debug("{} {}".format(idx, path))
            data = read_fixed_image(path)
            txn.put(str(idx), data, db=train_data)

        for idx, path in enumerate(train_label_list):
            logging.debug("{} {}".format(idx, path))
            data = read_fixed_label(path)
            txn.put(str(idx), data, db=train_label)

        for idx, path in enumerate(val_image_list):
            logging.debug("{} {}".format(idx, path))
            data = read_fixed_image(path)
            txn.put(str(idx), data, db=val_data)

        for idx, path in enumerate(val_label_list):
            logging.debug("{} {}".format(idx, path))
            data = read_fixed_label(path)
            txn.put(str(idx), data, db=val_label)